# Project: Monte Carlo Tree Search Mini Problem Set

- In this project, you will learn and implement the Monte Carlo Tree Search (MCTS) algoritm on the Tic Tac Toe game.

## Problem Set API

Before you write your very own MCTS bot, you need to get sped up on the API you will be using. But even before getting into the API, please run the following to import the API and some boilerplate code:

In [2]:
import algo
import sim
import random
import time
from tests import *
from game import *
import numpy as np

%load_ext autoreload
%autoreload 2
%matplotlib inline

### Default Policy
The first step is to implement the default policy, which plays through an entire game session. It chooses actions at random, applying them to the board until the game is over. It then returns the reward vector of the finished board.

<img src="figures/default-policy.png" alt="Default Policy Pseudocode" width="350px"/><br/>
<center><em><small>Browne, et al.</small></em></center>

**Note:** You can use `random.choice(my_list)` to select a random item from `my_list`

In [3]:
#######################################################################
# randomly picking moves to reach the end game
# Input: BOARD, the board that we want to start randomly picking moves
# Output: the reward vector when the game terminates
#######################################################################
def default_policy(board: ConnectFourBoard):
    while not board.is_terminal():  # while the board is not terminal
        move : ConnectFourAction = algo.random_algo(board) # randomly select a move
        board = move.apply(board)  # apply the move to the board
    return board.reward_vector()  # return the reward vector of the terminal board


In [4]:
test_default_policy(default_policy)

test passed
test passed
test passed


Tests passed!!

### Tree Policy

<img src="figures/tree-policy.png" alt="Tree Policy Pseudocode" width="250px"/><br/>
<center><em>Tree Policy Pseudocode (Browne, et al.)</em></center>


The tree policy performs a depth-first search of the tree using `best_child` and `expand`. If it encounters an unexpanded node it will return the expanded child of that node. Otherwise, it continues its search in the best child of the current node.

The `best_child` function find the best child node if a node is fully expanded. It also takes the exploitation constant as an argument.

The `expand` function expands a node that has unexpanded children. It must get all current children of the node and all possible children of the node then add one of the possible children to the node. It should then return this newly added child.

### Best Child
<img src="figures/best-child.png" alt="Best Child Pseudocode" width="400px"/><br/>
<center><em><small>Browne, et al.</small></em></center>

**Note:** For convenience, we've implemented a function that returns the heuristic inside the max operator. Look at the function `node.value(c)` for the `NODE` class API and save yourself the headache.

In [6]:
###########################################################
# get the best child from this node (using heuristic)
# Input: NODE, the node we want to find the best child of
#        C,    the exploitation constant
# Output: the child node
###########################################################
def best_child(node: Node, c) -> Node:
    if node.num_visits == 0:
        return random.choice(node.children)
    children:list[Node] = node.get_children()
    best_score = float('-inf')
    best_children = []
    
    # find the highest score among the children
    for child in children:
        exploitation = child.q_value() / child.get_num_visits()
        exploration = c * np.sqrt(2 * np.log(node.get_num_visits()) / child.get_num_visits())
        score = exploitation + exploration
        if score == best_score:
            best_children.append(child)
        elif score > best_score:
            best_score = score
            best_children = [child]
            
    return best_children[0]

In [7]:
test_best_child(best_child)

test passed


Tests passed!!

### Expand
<img src="figures/expand.png" alt="Expand Pseudocode" width="400px"/><br/>
<center><em><small>Browne, et al.</small></em></center>

In [51]:
###########################################################
# expand a node since it is not fully expanded
# Input: NODE, a node that want to be expanded
# Output: the child node
###########################################################
def expand(node: Node):
    board: ConnectFourBoard = node.get_board()
    children_actions = [child.get_action() for child in node.get_children()]
    unexplored_actions = [a for a in board.get_legal_actions() if a not in children_actions]

    action = random.choice(unexplored_actions)
    child_node = Node(action.apply(board), action, node)
    node.add_child(child_node)
    return child_node

In [84]:
test_expand(expand)

Tests passed!!

test passed


### Tree Policy
<img src="figures/tree-policy.png" alt="Tree Policy Pseudocode" width="250px"/><br/>
<center><em><small>Browne, et al.</small></em></center>

In [22]:
###########################################################
# heuristically search to the leaf level
# Input: NODE, a node that want to search down 
#        C, the exploitation value
# Output: the leaf node that we expand till
##########################################################
def tree_policy(node: Node, c):
    while not node.get_board().is_terminal():
        if not node.is_fully_expanded():
            # print("expand")
            return expand(node)
        else:
            # print("best_child")
            node = best_child(node, c)
    return node

In [101]:
test_tree_policy(tree_policy, expand, best_child)

test passed


Tests passed!!

### Backup
Now its time to make a way to turn the reward from `default_policy` into the information that `tree_policy` needs. `backup` should take the terminal state and reward from `default_policy` and proceed up the tree, updating the nodes on its path based on the reward.

<img src="figures/backup.png" alt="Backup Pseudocode" width="250px"/><br/>
<center><em><small>Browne, et al.</small></em></center>

In [105]:
##############################################################
# reward update for the tree after one simulation
# Input: NODE, the node that we want to backup from
#        REWARD_VECTOR, the reward vector of this exploration
# Output: nothing
##############################################################
def backup(node: Node, reward_vector):
    while node:
        node.num_visits += 1
        node.q += reward_vector[~node.get_player_id()]
        node = node.get_parent()

In [106]:
test_backup(backup)

test passed


Tests passed!!

### Search (UCT)
Time to put everything together! Keep running `tree_policy`, `default_policy`, and `backup` until you run out of time! Finally, return the best child's associated action.

<img src="figures/uct-search.png" alt="Search Pseudocode" width="300px"/><br/>
<center><em><small>Browne, et al.</small></em></center>

In [107]:
######################################################################
# monte carlo tree search algorithm using UCT heuristic
# Input: BOARD, the current game board
#        TIME_LIMIT, the time limit of the calculation in second
# Output: class Action represents the best action to take
####################################################################
def uct(board, time_limit):
    start_time = time.time()
    c = 1
    root = Node(board, None, None)
    while (time.time() - start_time) < time_limit:
        node_l = tree_policy(root, 2)
        rewards = default_policy(node_l.get_board())
        backup(node_l, rewards)
    return best_child(root, 0).get_action()

In [109]:
test_uct(uct) # this test takes 15-30 seconds

ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=1,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=3,row=3)
ConnectFourAction(color=B,col=2,row=2)
ConnectFourAction(color=B,col=2,row=2)
ConnectFourAction(color=B,col=4,row=1)
ConnectFourAction(color=B,col=2,row=2)
ConnectFourAction(color=B,col=2,row=2)
ConnectFourAction(color=B

Tests passed!!

## The Final Challenge
Time to show Stonn the power of human ingenuity! Win at least 9 out of 10 games to triumph!

In [ ]:
sim.run_final_test(uct)